In [148]:
from functions import *

In [149]:
def compute_corner_response(img):
    sobel_x=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
    sobel_y=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    I_x=cross_correlation_2d(img,sobel_x)
    I_y=cross_correlation_2d(img,sobel_y)
    window=np.ones((5,5))
    I_xx=cross_correlation_2d(I_x**2,window)
    I_yy=cross_correlation_2d(I_y**2,window)
    I_xy=cross_correlation_2d(I_x*I_y,window)
    k=0.04
    det=I_xx*I_yy-I_xy**2
    trace=I_xx+I_yy
    R=det-k*trace**2
    R=R.clip(0)
    R=(R-np.min(R))/(np.max(R)-np.min(R))
    return R

In [150]:
def get_corner_img(img_path):   
    img=get_gaussian_img(img_path,7,1.5)
    start=time.time()
    corner=compute_corner_response(img)
    print("corner response: {time.time()-start}")
    img_show(corner)
    directory="./result"
    if not os.path.exists(directory):
        os.makedirs(directory)
    corner = ((corner - corner.min()) / (corner.max() - corner.min()) * 255.).astype(np.uint8) 
    cv2.imwrite(f'./result/part_3_corner_raw_{img_path}',corner)
    return corner

In [ ]:
LENNA_IMG_PATH="lenna.png"
corner=get_corner_img(LENNA_IMG_PATH)
img=cv2.imread(LENNA_IMG_PATH,cv2.IMREAD_GRAYSCALE)
img=cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
img[corner>0.1]=[0,255,0]
img_show(img)
directory="./result"
if not os.path.exists(directory):
    os.makedirs(directory)
cv2.imwrite(f'./result/part_3_corner_bin_{LENNA_IMG_PATH}',img)

In [151]:
def get_green_img(img_path):
    corner=get_corner_img(img_path)
    img=cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    img[corner>0.1]=[0,255,0]
    img_show(img)
    directory="./result"
    if not os.path.exists(directory):
        os.makedirs(directory)
    cv2.imwrite(f'./result/part_3_corner_bin_{img_path}',img)

corner response: {time.time()-start}


True

In [152]:
def non_maximum_suppression_win(R):
    r,c=R.shape
    suppressed_R=np.zeros((r,c))
    for i in range(r):
        for j in range(c):
            if R[i,j]>0.1:
                rl,rr,cl,cr=max(0,i-5),min(r,i+6),max(0,j-5),min(c,j+6)
                window=R[rl:rr,cl:cr]
                maxi=np.max(window)
                if maxi==R[i,j]:
                    suppressed_R[i,j]=maxi
    return suppressed_R

In [153]:
LENNA_IMG_PATH="lenna.png"

img=get_gaussian_img(LENNA_IMG_PATH,7,1.5)
R=compute_corner_response(img)
suppressed_R=non_maximum_suppression_win(R)
backgroundImg=cv2.imread(LENNA_IMG_PATH,cv2.IMREAD_GRAYSCALE)
backgroundImg=cv2.cvtColor(backgroundImg,cv2.COLOR_GRAY2BGR)
for x,row in enumerate(suppressed_R):
    for y in np.where(row)[0]:
        cv2.circle(backgroundImg, (y,x), 5, (0,255,0),2)
directory="./result"
if not os.path.exists(directory):
    os.makedirs(directory)
cv2.imwrite(f'./result/part_3_corner_sup_{LENNA_IMG_PATH}',backgroundImg)
img_show(backgroundImg)

In [154]:
def get_circled_img(img_path):
    img=get_gaussian_img(img_path,7,1.5)
    R=compute_corner_response(img)
    start=time.time()
    suppressed_R=non_maximum_suppression_win(R)
    print(f"nms_window: {time.time()-start}")
    backgroundImg=cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    backgroundImg=cv2.cvtColor(backgroundImg,cv2.COLOR_GRAY2BGR)
    for x,row in enumerate(suppressed_R):
        for y in np.where(row)[0]:
            cv2.circle(backgroundImg, (y,x), 5, (0,255,0),2)
    directory="./result"
    if not os.path.exists(directory):
        os.makedirs(directory)
    cv2.imwrite(f'./result/part_3_corner_sup_{img_path}',backgroundImg)
    img_show(backgroundImg)